In [1]:
!head -n201 localdata/000001.csv | tail -n200 | awk -F',' '{print $1","$4}' > localdata/subdata.csv

In [2]:
!head -n6 localdata/subdata.csv

In [3]:
import pandas as pd

In [4]:
sr = pd.read_csv('localdata/subdata.csv', index_col=0, squeeze=True, header=None, names=['date', 'value'])
sr.index = pd.DatetimeIndex(sr.index)
sr = sr.reindex(pd.date_range(start='2017-08-09',end='2018-06-05', freq='D'))
sr.head(7)

2017-08-09    3275.5730
2017-08-10    3261.7494
2017-08-11    3208.5413
2017-08-12          NaN
2017-08-13          NaN
2017-08-14    3237.3602
2017-08-15    3251.2617
Freq: D, Name: value, dtype: float64

In [7]:
sr = sr.fillna(method='pad')
sr.head(7)

2017-08-09    3275.5730
2017-08-10    3261.7494
2017-08-11    3208.5413
2017-08-12    3208.5413
2017-08-13    3208.5413
2017-08-14    3237.3602
2017-08-15    3251.2617
Freq: D, Name: value, dtype: float64

In [43]:
from datetime import date, timedelta
def dataavg(data, today, day_before,day_delta=30):
    today = date(*today)
    n1 = day_delta / 2 - day_before
    n2 = -day_delta / 2 - day_before
    
    return data[today + timedelta(days=n2): today + timedelta(days=n1)].mean()

In [52]:
from dao.models import SZIndexStat

In [80]:
df = pd.DataFrame(sr)

In [90]:
all_insert_data = []
for i in range(df.shape[0]):
    _date = df.iloc[i].name.date()
    date_str = str(_date)
    value = float(df.iloc[i])
    one_line_data = {}
    one_line_data['avg_30_index'] = dataavg(sr, (_date.year, _date.month, _date.day), 30)
    one_line_data['avg_90_index'] = dataavg(sr, (_date.year, _date.month, _date.day), 90)
    one_line_data['avg_180_index'] = dataavg(sr, (_date.year, _date.month, _date.day), 180)
    one_line_data['date'] = date_str
    one_line_data['today_index_value'] = value
    all_insert_data.append(one_line_data)

SZIndexStat.insert_many(all_insert_data)
